Connected Solutions Math
=====================

Patrick Wagstrom &lt;160672+pridkett@users.noreply.github.com&gt;

June 2023

This is a really simple down and dirty notebook that I used to do the math in [my Weblog post about how much I get paid per kWh for electricity through Eversource's battery demand response program](https://patrick.wagstrom.net/weblog/2023/06/23/powerwall-and-battery-demand-response/).

In [1]:
import pandas as pd
import requests
from io import StringIO
import time

In [2]:
start_time = time.time()

paycheck_amount = 1889.42

start_date = '2022-06-01T04:00:00Z'
stop_date = '2022-10-01T04:00:00Z'

time_grouping = '1h'
timezone = 'America/New_York'

"""select * from system where temperature = 0 and time > '2023-05-01T00:00:00Z' order by time asc limit 100;"""

influxdb_url = 'http://[YOURINFULXDBSERVER]:8086/query'

params = {
    'db': 'powerwall',
    'q': f"""SELECT integral("to_grid") / 1000 / 3600 as "kWh" FROM "autogen"."http" WHERE time > '{start_date}' and time < '{stop_date}' GROUP BY time({time_grouping}) tz('{ timezone }')""",
    'epoch': 's'
}

headers = {
    'Accept': 'application/csv',
}

# Make the GET request
response = requests.get(influxdb_url, params=params, headers=headers)

# Make sure the request was successful
response.raise_for_status()

# Get the data as a CSV string
data = response.text

# Convert to a DataFrame
df = pd.read_csv(StringIO(data))

finish_time = time.time()
print(f"{df.size} rows fetched in {finish_time-start_time:.2f} seconds")

9900 rows fetched in 0.91 seconds


In [3]:
df = df.drop(["name", "tags"], axis=1)
df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
df['time'] = df['time'].dt.tz_convert('America/New_York')

In [4]:
filtered_df = df[(df['time'].dt.hour >= 14) & (df['time'].dt.hour <= 19)]
filtered_df = filtered_df[filtered_df['kWh'] > 5]
filtered_df

,time,kWh
405,2022-07-06 17:00:00-04:00,18.290685
406,2022-07-06 18:00:00-04:00,17.100467
453,2022-07-08 17:00:00-04:00,15.295984
454,2022-07-08 18:00:00-04:00,15.170017
525,2022-07-11 17:00:00-04:00,17.961609
...,...,...
1700,2022-08-29 17:00:00-04:00,13.589776
1701,2022-08-29 18:00:00-04:00,12.101548
1723,2022-08-30 16:00:00-04:00,14.224240
1724,2022-08-30 17:00:00-04:00,5.672601


In [5]:
unique_days = filtered_df['time'].dt.date.nunique()
kWh_sum = filtered_df['kWh'].sum()
price_kWh = paycheck_amount / kWh_sum
print(f"""Total kWh: { int(kWh_sum) } at ${price_kWh:.2f}/kWh""")
price_event = paycheck_amount / unique_days
kWh_event = kWh_sum / unique_days
print(f"""Total Events: { unique_days} at ${price_event:.2f}/event and {kWh_event:.2f}kWh/Event""")

Total kWh: 1029 at $1.84/kWh
Total Events: 41 at $46.08/event and 25.11kWh/Event
